In [12]:
"""
By Gideon Kassa
June 14, 2024

Liu Group
Dartmouth College
"""

import numpy as np
from scipy.optimize import curve_fit, minimize
import matplotlib.pyplot as plt
import sympy as sp
from sklearn.metrics import r2_score

In [4]:
with open('C8_514_aSiO2_200c_R_T_A.txt', 'r+') as f:
    full_file = f.read()

Eg, R, T, A = np.array([i.split('\t') for i in full_file.split('\n')], dtype=np.float32).T

wavelengths = 1240/Eg  # nm
transmission = T/100  # 0 to 1
reflection = R/100  # 0 to 1
thickness = 460  # nm

In [5]:
with open('n_real.txt', 'r+') as f:
    full_file = f.read()

dft_Eg, dft_n = np.array([i.split(' ') for i in full_file.split('\n')[2:-1]], dtype=np.float32).T
dft_wl = 1240/dft_Eg

In [ ]:
# below is method 1, the extensive approach
n1, k1, n2, k2, n3, k3, n4, k4, d1, d2, d3, d4, wl, R = sp.symbols('n1 k1 n2 k2 n3 k3 n4 k4 d1 d2 d3 d4 wl, R')


beta_2 = 2 * np.pi * d2 / wl
beta_3 = 2 * np.pi * d3 / wl

n_k1 = n1 + 1j * k1
n_k2 = n2 + 1j * k2
n_k3 = n3 + 1j * k3
n_k4 = n4 + 1j * k4

kx1 = n_k1 * 2 * np.pi / wl
kx2 = n_k2 * 2 * np.pi / wl
kx3 = n_k3 * 2 * np.pi / wl
kx4 = n_k4 * 2 * np.pi / wl

M_I1 = sp.Matrix(
    [[(n_k1 + n_k2)/(2 * n_k1), (n_k1 - n_k2)/(2 * n_k1)],
     [(n_k1 - n_k2)/(2 * n_k1), (n_k1 + n_k2)/(2 * n_k1)]])

M_P = sp.Matrix(
    [[sp.exp(-1j * n_k2 * beta_2), 0],
     [0, sp.exp(1j * n_k2 * beta_2)]
])

M_I2 = sp.Matrix(
    [[(n_k2 + n_k3)/(2 * n_k2), (n_k2 - n_k3)/(2 * n_k2)],
     [(n_k2 - n_k3)/(2 * n_k2), (n_k2 + n_k3)/(2 * n_k2)]])

M_P2 = sp.Matrix(
    [[sp.exp(-1j * n_k3 * beta_3), 0],
     [0, sp.exp(1j * n_k3 * beta_3)]])

M_I3 = sp.Matrix(
    [[(n_k3 + n_k1)/(2 * n_k3), (n_k3 - n_k1)/(2 * n_k3)],
     [(n_k3 - n_k1)/(2 * n_k3), (n_k3 + n_k1)/(2 * n_k3)]])


M = M_I1 * M_P * M_I2 * M_P2 * M_I3


T = abs(1/M[0, 0])**2
R = abs(M[1, 0]/M[0, 0])**2

T = sp.lambdify([n1, k1, n2, k2, n3, k3, n4, k4, d1, d2, d3, d4, wl], T)
R = sp.lambdify([n1, k1, n2, k2, n3, k3, n4, k4, d1, d2, d3, d4, wl], R)

wl = 500

n1 = 1
k1 = 0

n2 = 3.66769
k2 = 0.27805

n3 = 1.46
k3 = 0

n4 = 1
k4 = 0

d1 = 0
d2 = 465
d3 = 1e6
d4 = 0


print(f"transmission = {T(n1, k1, n2, k2, n3, k3, n4, k4, d1, d2, d3, d4, wl)}")
print(f"reflection = {R(n1, k1, n2, k2, n3, k3, n4, k4, d1, d2, d3, d4, wl)}")

In [ ]:
# below is method 2, the paper fitting
wl = 500

n1 = 1.5
k1 = 0

n2 = 3.66769
k2 = 0.27805

n3 = 1
k3 = 0

n4 = 1
k4 = 0

d1 = 0
d2 = 465
d3 = 0
d4 = 0

def jreftran_rt(wavelength, thicknesses, refractive_indices, incidence_angle, polarization):
    Z0 = 376.730313
    Y = refractive_indices / Z0
    g = 1j * 2 * np.pi * refractive_indices / wavelength

    t = np.arcsin(refractive_indices[0] / refractive_indices * np.sin(incidence_angle))
    ct = np.sqrt(1 - (refractive_indices[0] / refractive_indices * np.sin(incidence_angle))**2)

    if polarization == 0:
        eta = Y * ct
    else:
        eta = Y / ct

    delta = 1j * g * thicknesses * ct
    M = np.zeros((2, 2, len(thicknesses)), dtype=complex)

    for j in range(len(thicknesses)):
        M[0, 0, j] = np.cos(delta[j])
        M[0, 1, j] = 1j / eta[j] * np.sin(delta[j])
        M[1, 0, j] = 1j * eta[j] * np.sin(delta[j])
        M[1, 1, j] = np.cos(delta[j])

    M_t = np.array([[1, 0], [0, 1]], dtype=complex)
    for j in range(1, len(thicknesses) - 1):
        M_t = np.matmul(M_t, M[:, :, j])

    r = (eta[0] * (M_t[0, 0] + M_t[0, 1] * eta[-1]) - (M_t[1, 0] + M_t[1, 1] * eta[-1])) / (eta[0] * (M_t[0, 0] + M_t[0, 1] * eta[-1]) + (M_t[1, 0] + M_t[1, 1] * eta[-1]))
    t = 2 * eta[0] / (eta[0] * (M_t[0, 0] + M_t[0, 1] * eta[-1]) + (M_t[1, 0] + M_t[1, 1] * eta[-1]))

    R = abs(r)**2
    T = np.real(eta[-1] / eta[0]) * abs(t)**2
    A = 4 * np.real(eta[0] * (M_t[0, 0] + M_t[0, 1] * eta[-1]) * np.conj(M_t[1, 0] + M_t[1, 1] * eta[-1]) - eta[-1]) / abs(eta[0] * (M_t[0, 0] + M_t[0, 1] * eta[-1]) + (M_t[1, 0] + M_t[1, 1] * eta[-1]))**2

    return r, t, R, T, A

R = lambda n1, k1, n2, k2, n3, k3, n4, k4, d1, d2, d3, d4, wl: jreftran_rt(wl, np.array([d1, d2, d3]), np.array([n1 + 1j * k1, n2 + 1j * k2, n3 + 1j * k3]), 0, 0)[2]
T = lambda n1, k1, n2, k2, n3, k3, n4, k4, d1, d2, d3, d4, wl: jreftran_rt(wl, np.array([d1, d2, d3]), np.array([n1 + 1j * k1, n2 + 1j * k2, n3 + 1j * k3]), 0, 0)[3]
RT = lambda n1, k1, n2, k2, n3, k3, n4, k4, d1, d2, d3, d4, wl: jreftran_rt(wl, np.array([d1, d2, d3]), np.array([n1 + 1j * k1, n2 + 1j * k2, n3 + 1j * k3]), 0, 0)
print(f"transmission = {T(n1, k1, n2, k2, n3, k3, n4, k4, d1, d2, d3, d4, wl)}")
print(f"reflection = {R(n1, k1, n2, k2, n3, k3, n4, k4, d1, d2, d3, d4, wl)}")


In [ ]:
# below is my variation of method 2
n1, k1, n2, k2, n3, k3, n4, k4, d1, d2, d3, d4, wl, R = sp.symbols('n1 k1 n2 k2 n3 k3 n4 k4 d1 d2 d3 d4 wl, R')

n_k1 = n1 + 1j * k1
n_k2 = n2 + 1j * k2
n_k3 = n3 + 1j * k3
n_k4 = n4 + 1j * k4

Z0 = 376.730313


g1 = 1j * 2 * sp.pi * n_k1 / wl
delta1 = 1j * g1 * d1
Y1 = n_k1/Z0
eta1 = Y1


g2 = 1j * 2 * sp.pi * n_k2 / wl
delta2 = 1j * g2 * d2
Y2 = n_k2/Z0
eta2 = Y2


g3 = 1j * 2 * sp.pi * n_k3 / wl
delta3 = 1j * g3 * d3
Y3 = n_k3/Z0
eta3 = Y3


g4 = 1j * 2 * sp.pi * n_k4 / wl
delta4 = 1j * g4 * d4
Y4 = n_k4/Z0
eta4 = Y4

ds = [d1, d2, d3, d4]
gs = [g1, g2, g3, g4]
deltas = [delta1, delta2, delta3, delta4]
etas = [eta1, eta2, eta3, eta4]

M = sp.Matrix([[1, 0], [0, 1]])
index = 0
for d, g, delta, eta in zip(ds, gs, deltas, etas):

    if 0 < index < 2:

        Mi = sp.Matrix([
            [sp.cos(delta), 1j*sp.sin(delta)/eta],
            [1j*eta*sp.sin(delta), sp.cos(delta)]
        ])

        M *= Mi
        print(d)

    index += 1

eta0 = eta1
etaN = eta3

r = (eta0*(M[0, 0] + M[0, 1]*etaN) - (M[1, 0] + M[1, 1]*etaN)) / (eta0*(M[0, 0] + M[0, 1]*etaN) + (M[1, 0] + M[1, 1]*etaN))
t = 2 * eta0 / (eta0*(M[0, 0] + M[0, 1]*etaN) + (M[1, 0] + M[1, 1]*etaN))

R = abs(r)**2
T = sp.re(etaN/eta0) * abs(t)**2


R = sp.lambdify([n1, k1, n2, k2, n3, k3, n4, k4, d1, d2, d3, d4, wl], R)
T = sp.lambdify([n1, k1, n2, k2, n3, k3, n4, k4, d1, d2, d3, d4, wl], T)

wl = 500

n1 = 1.4
k1 = 0

n2 = 3.66769
k2 = 0.27805

n3 = 1
k3 = 0

n4 = 1
k4 = 0

d1 = 0
d2 = 460
d3 = 0
d4 = 0

print(f"transmission = {T(n1, k1, n2, k2, n3, k3, n4, k4, d1, d2, d3, d4, wl)}")
print(f"reflection = {R(n1, k1, n2, k2, n3, k3, n4, k4, d1, d2, d3, d4, wl)}")
print(f"combined = {T(n1, k1, n2, k2, n3, k3, n4, k4, d1, d2, d3, d4, wl) + R(n1, k1, n2, k2, n3, k3, n4, k4, d1, d2, d3, d4, wl)}")


In [ ]:
# below is the manual fitting for the above methods

n_air = 1
k_air = 0
k_substrate = 0
d = 500

n = []
k = []

calc_r = []
calc_t = []

st = 1
ed = 1000

for indx, wavelength_i, R_i, T_i in zip(range(ed-st), wavelengths[st:ed], reflection[st:ed], transmission[st:ed]):
    x = wavelength_i/1000
    n_substrate_i = (1+0.9310/(1-(0.079/x)**2)+0.1735/(1-(0.130/x)**2)+2.1121/(1-(14.918/x)**2))**.5

    n_film_i = dft_n[np.argmin(np.abs(wavelength_i - dft_wl))]
    k_film_i = 0.1

    for itr in range(30):


        diff = 0

        ks = np.linspace(0, 2, 50)
        Ts = []
        for k_guess in ks:

            T_guess = T(n1=1.45, k1=0, n2=n_film_i, k2=k_guess, n3=1, k3=0, n4=1, k4=0, d1=0, d2=d, d3=0, d4=0, wl=wavelength_i)
            Ts.append(np.abs(T_guess - T_i))
        diff += np.abs(k_film_i - ks[np.argmin(Ts)])
        k_film_i = ks[np.argmin(Ts)]

        ns = np.linspace(1, 5, 50)
        Rs = []
        for n_guess in ns:

            R_guess = T(n1=1.45, k1=0, n2=n_guess, k2=k_film_i, n3=1, k3=0, n4=1, k4=0, d1=0, d2=d, d3=0, d4=0, wl=wavelength_i)
            Rs.append(np.abs(R_guess - T_i))

        diff += np.abs(n_film_i - ns[np.argmin(Rs)])
        n_film_i = ns[np.argmin(Rs)]



        if diff < 0.0000001:
            break
        elif itr == 10 - 1:

            if indx == 0:
                n_film_i = dft_n[np.argmin(np.abs(wavelength_i - dft_wl))]
            else:
                n_film_i = n[indx - 1]
            print('not converged')

    n.append(n_film_i)
    k.append(k_film_i)

    if wavelength_i % 50 == 0:
        print(f'calculating for wavelength {wavelength_i} nm')

plt.plot(wavelengths[st:ed], k, label='Refractive Index (n)')
plt.plot(wavelengths[st:ed], n, label='Refractive Index (n)')

print(n, k)

In [ ]:
# below is fitting using Nelder-Mead optimization

use_approach_1 = False
def objective(x, n, wavelength, measured_R, measured_T, d, n_subs):

    if use_approach_1:
        r = R(1, 0, n, x[0], n_subs, 0, 1, 0, 0, thickness, 0, 0, wavelength)
        t = T(1, 0, n, x[0], n_subs, 0, 1, 0, 0, thickness, 0, 0, wavelength)

        T_calc = r
        R_calc = t

        return (T_calc - measured_T)**2 #+ (R_calc - measured_R)**2

    else:
        rt_c = RT(1.4, 0, n, x[0], 1, 0, 1, 0, 0, d, 0, 0, wavelength)

        T_calc = rt_c[3]
        R_calc = rt_c[2]

        return (T_calc - measured_T)**2 #+ (R_calc - measured_R)**2

def objective_n(x, k, wavelength, measured_R, measured_T, d, n_subs):

    if use_approach_1:

        r = R(1, 0, x[0], k, n_subs, 0, 1, 0, 0, thickness, 0, 0, wavelength)
        t = T(1, 0, x[0], k, n_subs, 0, 1, 0, 0, thickness, 0, 0, wavelength)

        T_calc = t
        R_calc = r

        return (T_calc - measured_T)**2 #+ (R_calc - measured_R)**2

    else:

        rt_c = RT(1.4, 0, x[0], k, 1, 0, 1, 0, 0, d, 0, 0, wavelength)

        T_calc = rt_c[3]
        R_calc = rt_c[2]

        return (T_calc - measured_T)**2 #+ (R_calc - measured_R)**2


k_guess = 0.3
thickness = 465

k = []
n = []
for wavelength_i, R_i, T_i in zip(wavelengths, reflection, transmission):
    x = wavelength_i/1000
    n_substrate = (1+0.9310/(1-(0.079/x)**2)+0.1735/(1-(0.130/x)**2)+2.1121/(1-(14.918/x)**2))**.5

    n_guess = dft_n[np.argmin(np.abs(wavelength_i - dft_wl))]


    for idxx in range(20):
        initial_guess = [0]
        popt = minimize(objective, initial_guess, args=(n_guess, wavelength_i, R_i, T_i, thickness, n_substrate), method='Nelder-Mead')
        diff = np.abs(popt.x[0] - k_guess)
        k_guess = popt.x[0]

        initial_guess = [n_guess]
        popt = minimize(objective_n, initial_guess, args=(k_guess, wavelength_i, R_i, T_i, thickness, n_substrate), method='Nelder-Mead')
        diff += np.abs(popt.x[0] - n_guess)
        n_guess = popt.x[0]

        if diff == 0:
            break
        if idxx == 19:
            print('not converged')

    # prevent crazy ks
    if len(k)>0:
        if abs(k_guess - k[-1])>2:
            k.append(k[-1])

        else:
            k.append(k_guess)
    else:
        k.append(k_guess)
    n.append(n_guess)

plt.plot(wavelengths, n, label='n')
plt.plot(wavelengths, k, label='k')
plt.plot(dft_wl, dft_n, 'r-', label='DFT')
plt.xlim(min(wavelengths), max(wavelengths))

plt.xlabel('Wavelength (nm)')
plt.legend()
plt.show()

In [ ]:
ts = []
for ns, ks, wl in zip(n, k, wavelengths):
    x = wavelength_i/1000
    n_substrate_i = (1+0.9310/(1-(0.079/x)**2)+0.1735/(1-(0.130/x)**2)+2.1121/(1-(14.918/x)**2))**.5

    ts.append(RT(1, 0, ns, ks, n_substrate_i, 0, 1, 0, 0, thickness, 0, 0, wl)[2])
plt.plot(wavelengths, ts, label='Calculated Reflection')
plt.plot(wavelengths, reflection, 'g-', label='Measured Reflection')
plt.xlabel('Wavelength (nm)')
plt.legend()

In [ ]:
# below is fitting using Cauchy's equation

def func(wl, a, b, c, d, e, f):

    ts = []

    thickness = 465

    for l in wl:
        x = l/1000

        n = a + (b/(l**2)) + (c/(l**4))
        k = d + (e/(l**2)) + (f/(l**4))

        n_subs = (1+0.9310/(1-(0.079/x)**2)+0.1735/(1-(0.130/x)**2)+2.1121/(1-(14.918/x)**2))**.5
        # rt = RT(n_subs, 0, n, k, 1, 0, 1, 0, 0, thickness, 0, 0, l)
        r = R(1, 0, n, k, n_subs, 0, 1, 0, 0, thickness, 0, 0, l)
        t = T(1, 0, n, k, n_subs, 0, 1, 0, 0, thickness, 0, 0, l)
        ts.append(t)

    return ts

popt_f, pcov_f = curve_fit(func, wavelengths, transmission, p0=[1, 1e4, 1e9, 1e-5, 1e4, 1e9], maxfev=80_000)
intensity_pred_f = func(wavelengths, *popt_f)

r2_f = r2_score(transmission, intensity_pred_f)
a, b, c, d, e, f = popt_f[:6]
print(f"A = {popt_f[0]}, B = {popt_f[1]}, c = {popt_f[2]}, d = {popt_f[3]}, e = {popt_f[4]}, f = {popt_f[5]}, R^2 = {r2_f}")

n = []
k = []
for l in wavelengths:
    n.append(a + (b/(l**2)) + (c/(l**4)))
    k.append(d + (e/(l**2)) + (f/(l**4)))

plt.plot(wavelengths, n, label='Fitting')
plt.plot(wavelengths, k, label='Fitting')
plt.plot(dft_wl, dft_n, 'r-', label='DFT')
plt.xlim(min(wavelengths), max(wavelengths))

plt.xlabel('Wavelength (nm)')
plt.ylabel('Refractive Index (n)')

plt.show()